# Virtual Home Planner using LAPKT

The following provides a complete instruction on generating actions given Virtualhome environment intial setting defined using a pddl file. Please see 'example_problem.pddl' to know how these problem files are defined.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1_FMnK6rmCjVCtQbj18qbuhxJGrIvA7Up?usp=sharing)

# Setup PDDLMultiAgent Planner Directory

In [1]:
!git -c core.symlinks=true clone --recurse-submodules https://github.com/xavierpuigf/PDDLMultiAgent
%cd PDDLMultiAgent

Cloning into 'PDDLMultiAgent'...
remote: Enumerating objects: 197, done.
remote: Total 197 (delta 0), reused 0 (delta 0), pack-reused 197
Receiving objects: 100% (197/197), 210.27 KiB | 10.51 MiB/s, done.
Resolving deltas: 100% (96/96), done.
Submodule 'LAPKT-public' (https://github.com/LAPKT-dev/LAPKT-public.git) registered for path 'LAPKT-public'
Cloning into '/content/PDDLMultiAgent/LAPKT-public'...
remote: Enumerating objects: 11594, done.        
remote: Counting objects: 100% (1175/1175), done.        
remote: Compressing objects: 100% (332/332), done.        
remote: Total 11594 (delta 898), reused 843 (delta 843), pack-reused 10419        
Receiving objects: 100% (11594/11594), 15.03 MiB | 14.08 MiB/s, done.
Resolving deltas: 100% (6977/6977), done.
Submodule path 'LAPKT-public': checked out 'af757138d0b4d903e35e5946b3b88e8f8d64e976'
/content/PDDLMultiAgent


# Build LAPKT

In [2]:
!sudo apt-get install xutils-dev
!sudo apt install flex 
!sudo apt install bison
!python -m pip install scons

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  xutils-dev
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 239 kB of archives.
After this operation, 1,503 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 xutils-dev amd64 1:7.7+5ubuntu1 [239 kB]
Fetched 239 kB in 2s (140 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package xutils-dev.
(Reading database ... 155455 files and direc

In [3]:
%cd LAPKT-public
!scons

/content/PDDLMultiAgent/LAPKT-public
scons: Reading SConscript files ...
  File "/content/PDDLMultiAgent/LAPKT-public/SConstruct", line 31

    if src_objs is None : print "src_objs is None"

                                                 ^

SyntaxError: Missing parentheses in call to 'print'. Did you mean print("src_objs is None")?



In [4]:
%cd ./external/libff
!make clean
!make depend
!make

/content/PDDLMultiAgent/LAPKT-public/external/libff
rm -f *.o *.bak *~ *% core *_pure_p9_c0_400.o.warnings test_lib \
        \#*\#  scan-fct_pddl.tab.c scan-ops_pddl.tab.c scan-probname.tab.c lex.fct_pddl.c lex.ops_pddl.c 
makedepend -- main.c memory.c output.c parse.c expressions.c inst_pre.c inst_easy.c inst_hard.c inst_final.c relax.c search.c utility.c  libff.c memory.c output.c parse.c expressions.c inst_pre.c inst_easy.c inst_hard.c inst_final.c relax.c search.c utility.c scan-fct_pddl.tab.c scan-ops_pddl.tab.c scan-probname.tab.c lex.fct_pddl.c lex.ops_pddl.c  test_main.cxx
makedepend: warning:  main.c (reading /usr/include/stdlib.h, line 25): cannot find include file "bits/libc-header-start.h"
	not in /usr/include/bits/libc-header-start.h
makedepend: warning:  main.c (reading /usr/include/stdlib.h, line 31): cannot find include file "stddef.h"
	not in /usr/include/stddef.h
makedepend: warning:  main.c (reading /usr/include/stdlib.h, line 55): cannot find include file "bits/flo

In [5]:
%cd ../..

/content/PDDLMultiAgent/LAPKT-public


In [6]:
%cd ./planners/siw_plus-then-bfs_f-ffparser
!scons

/content/PDDLMultiAgent/LAPKT-public/planners/siw_plus-then-bfs_f-ffparser
scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
g++ -o src/main.o -c -Wall -std=c++11 -O3 -DNDEBUG -I/content/PDDLMultiAgent/LAPKT-public/src -I/content/PDDLMultiAgent/LAPKT-public/include -I/content/PDDLMultiAgent/LAPKT-public/interfaces/agnostic -I/content/PDDLMultiAgent/LAPKT-public/interfaces/ff-wrapped -I/content/PDDLMultiAgent/LAPKT-public/include -I/content/PDDLMultiAgent/LAPKT-public/external -I/content/PDDLMultiAgent/LAPKT-public/interfaces/agnostic -I/content/PDDLMultiAgent/LAPKT-public/interfaces/ff-wrapped -Isrc src/main.cxx
g++ -o /content/PDDLMultiAgent/LAPKT-public/src/bit_array.o -c -Wall -std=c++11 -O3 -DNDEBUG -I/content/PDDLMultiAgent/LAPKT-public/src -I/content/PDDLMultiAgent/LAPKT-public/include -I/content/PDDLMultiAgent/LAPKT-public/interfaces/agnostic -I/content/PDDLMultiAgent/LAPKT-public/interfaces/ff-wrapped -I/content/PDDLMultiAgent

In [7]:
%cd ../../external/libff
!make clean
!make depend
!make

/content/PDDLMultiAgent/LAPKT-public/external/libff
rm -f *.o *.bak *~ *% core *_pure_p9_c0_400.o.warnings test_lib \
        \#*\#  scan-fct_pddl.tab.c scan-ops_pddl.tab.c scan-probname.tab.c lex.fct_pddl.c lex.ops_pddl.c 
makedepend -- main.c memory.c output.c parse.c expressions.c inst_pre.c inst_easy.c inst_hard.c inst_final.c relax.c search.c utility.c  libff.c memory.c output.c parse.c expressions.c inst_pre.c inst_easy.c inst_hard.c inst_final.c relax.c search.c utility.c scan-fct_pddl.tab.c scan-ops_pddl.tab.c scan-probname.tab.c lex.fct_pddl.c lex.ops_pddl.c  test_main.cxx
makedepend: warning:  main.c (reading /usr/include/stdlib.h, line 25): cannot find include file "bits/libc-header-start.h"
	not in /usr/include/bits/libc-header-start.h
makedepend: warning:  main.c (reading /usr/include/stdlib.h, line 31): cannot find include file "stddef.h"
	not in /usr/include/stddef.h
makedepend: warning:  main.c (reading /usr/include/stdlib.h, line 55): cannot find include file "bits/flo

# Sample Run a planner

In [8]:
root_dir = "/content/PDDLMultiAgent"
%cd $root_dir

/content/PDDLMultiAgent


## Implement customize siw-then-bfsf planner wrapper

The planner wrapper from the repo does not work as it depends on a symlink on github. 

In [9]:
import subprocess
import os
import pdb
import utils_env_parser
import urllib.request as urllibreq
import urllib
import argparse
import json, sys
import time

In [10]:
planner_pth = os.path.join(root_dir, 'LAPKT-public', 'planners', 'siw_plus-then-bfs_f-ffparser' ,'siw-then-bfsf')

def convert_to_virtualhome_program(action_list):
    action_list = [x[1:-2].split() for x in action_list]
    action_list_str = []
    for action_instr in action_list:
        if len(action_instr) == 0:
            continue
        action_item = '[{}]'.format(action_instr[0].upper())
        if len(action_instr) > 1:
            objects_item = ['<{}> ({})'.format('_'.join(l.split('_')[:-1]), l.split('_')[-1]) for l in action_instr[1:]]
        action_list_str.append('{} {}'.format(action_item, ' '.join(objects_item)))
    return action_list_str

def planner(domain_name, problem_name, file_out, planner_dir = planner_pth):
    start = time.time()
    stats_dir = '/'.join(file_out.split('/')[:-3]) + '/stats/' 
    file_stats = stats_dir + file_out.split('/')[-1] + '_stats.txt'
    os.makedirs(stats_dir, exist_ok=True)
    with open(file_stats, 'w+') as outfile:
        proc = subprocess.call([planner_pth, '--domain', domain_name, '--problem', problem_name, '--output', file_out], stdout=outfile)
    with open(file_out, 'r') as f:
        program = f.readlines()
        if len(program) > 0 and len(program[-1]) == 0:
            program = program[:-1]
    program = [x+'\n' for x in convert_to_virtualhome_program(program)]
    if len(program) > 0:
        with open(file_out, 'w+') as f:
            f.writelines(program)
        print("Successfully executed the planner.")
    else:
        os.remove(file_stats)
        os.remove(file_out)
        print("Planner has no output.")
    end = time.time()
    print("Time used: {}".format(end-start))


### Sample run
Replace example_problem/example_problem.pddl with the following content.


(define (problem setuptable1)
(:domain virtualhome)
(:objects
    plate_1, plate_2, plate_3, table_4 drawer_5 - object
    kitchen_6 livingroom_7 - room
    char_0 - character
    char_8 - character
)
(:init
    (= (objects_grabbed) 0)
    (container drawer_5)
    (surface table_4)
    (grabable plate_1)
    (grabable plate_2)
    (grabable plate_3)
    (inside plate_1 drawer_5)
    (inside plate_2 drawer_5)
    (inside plate_3 drawer_5)
    (inside drawer_5 kitchen_6)
    (inside table_4 livingroom_7)
    (inside char_0 livingroom_7)
    (inside char_8 kitchen_6)
)
(:goal
    (and (ontop plate_1 table_4) (ontop plate_3 table_4) (inside char_8 livingroom_7) (inside char_0 kitchen_6))
)
)

In [ ]:
#! ./LAPKT-public/planners/siw_plus-then-bfs_f-ffparser/siw-then-bfsf --domain domain.pddl --problem example_problem/example_problem.pddl --output file_out

In [63]:
domain_name='./domain.pddl'
problem_name='./example_problem/example_problem.pddl'
output_dir = "./example_out"
planner(domain_name=domain_name, problem_name=problem_name, file_out=output_dir)

Successfully executed the planner.
Time used: 0.014673233032226562


In [58]:
#printout result
f = open('./example_out')
for line in f:
    print(line)

[WALK] <> (KITCHEN) <> (CHAR1)

[WALK] <> (DRAWER) <> (CHAR1)

[OPEN] <> (DRAWER) <> (CHAR1)

[WALK] <> (PLATE3) <> (CHAR1)

[GRAB] <> (PLATE3) <> (CHAR1)

[WALK] <> (LIVINGROOM) <> (CHAR1)

[WALK] <> (TABLE) <> (CHAR1)

[PUT] <> (PLATE3) <> (TABLE) <> (CHAR1)

[WALK] <> (KITCHEN) <> (CHAR1)

[WALK] <> (DRAWER) <> (CHAR1)

[WALK] <> (PLATE1) <> (CHAR1)

[GRAB] <> (PLATE1) <> (CHAR1)

[WALK] <> (LIVINGROOM) <> (CHAR1)

[WALK] <> (TABLE) <> (CHAR1)

[PUT] <> (PLATE1) <> (TABLE) <> (CHAR1)



# Customized under Here

In [11]:
import json
## add tasks
## remember to upload goals.json


tasks = open('/content/PDDLMultiAgent/goals.json')
tasks = json.load(tasks)

In [12]:
tasks

{'(and  (ontop bananas_317 bathroomcabinet_49) (inside clothespants_133 clothespile_395) (grabbed character_1 cutleryfork_280) (grabbed character_2 clothespile_395))': [' (ontop bananas_317 bathroomcabinet_49)',
  ' (inside clothespants_133 clothespile_395)',
  '(grabbed character_1 cutleryfork_280)',
  '(grabbed character_2 clothespile_395)'],
 '(and  (ontop bananas_317 bathroomcabinet_49) (inside mug_196 clothespile_396) (grabbed character_1 clothespile_398) (grabbed character_2 cutleryfork_280))': [' (ontop bananas_317 bathroomcabinet_49)',
  ' (inside mug_196 clothespile_396)',
  '(grabbed character_1 clothespile_398)',
  '(grabbed character_2 cutleryfork_280)'],
 '(and  (ontop bananas_317 chair_109) (inside clothespants_141 kitchencabinet_235) (grabbed character_1 hanger_121) (grabbed character_2 hanger_393))': [' (ontop bananas_317 chair_109)',
  ' (inside clothespants_141 kitchencabinet_235)',
  '(grabbed character_1 hanger_121)',
  '(grabbed character_2 hanger_393)'],
 '(and  (

In [13]:
!mkdir -p sample_env/input_envs
!mkdir -p sample_env/out_problems
#place simplified_sample.json under input_envs

In [14]:
!python env_parser.py --folder_env sample_env/input_envs --folder_out sample_env/out_problems

In [37]:
list(tasks.keys())[40]

'(and  (ontop mug_196 plate_279) (inside clothespile_396 clothespile_396) (grabbed character_1 clothespile_144) (grabbed character_2 cutleryfork_280))'

In [19]:
tasks.keys()

dict_keys(['(and  (ontop clothespants_387 kitchencabinet_235) (inside bananas_317 clothespile_413) (grabbed character_1 hanger_196) (grabbed character_2 clothespants_387))', '(and  (ontop clothespants_387 towelrack_43) (inside hanger_115 clothespile_402) (grabbed character_1 plate_195) (grabbed character_2 cutleryfork_280))', '(and  (ontop hanger_144 bookshelf_371) (inside bananas_316 clothespile_153) (grabbed character_1 clothespile_109) (grabbed character_2 cutleryknife_281))', '(and  (ontop cereal_335 kitchencabinet_237) (inside hanger_121 clothespile_398) (grabbed character_1 bananas_153) (grabbed character_2 clothespile_153))', '(and  (ontop mug_196 plate_279) (inside hanger_121 clothespile_402) (grabbed character_1 plate_195) (grabbed character_2 plate_195))', '(and  (ontop cutleryfork_280 bookshelf_371) (inside hanger_144 clothespile_402) (grabbed character_1 bananas_195) (grabbed character_2 cutleryfork_280))', '(and  (ontop plate_279 chair_109) (inside hanger_384 clothespile_3

In [15]:
content = ""
outputfile = "./sample_env/out_problems/file_0.pddl"
goal = '(and (inside clothespants_387 clothespile_413) (inside plate_195 kitchencabinet_235) (grabbed character_1 towel_68) (grabbed character_2 cereal_335))'
flag = False

f = open('/content/PDDLMultiAgent/sample_env/out_problems/file_0.pddl', 'r')
for line in f:
    #print(line)
    content += line if not flag else ""
    if line.startswith('(:goal'):
        flag = True 
    if flag and line.startswith(')'):
        flag = False
        content += goal + '\n'
        content += ')\n'
        
wf = open(outputfile, 'w')
wf.write(content)
wf.close()
domain_name='./domain.pddl'
problem_name='./sample_env/out_problems/file_0.pddl'
output_dir = "./example_out"
! ./LAPKT-public/planners/siw_plus-then-bfs_f-ffparser/siw-then-bfsf --domain $domain_name --problem $problem_name --output $output_dir



task contains conditional effects. turning off state domination.

 --- OK.
 Match tree built with 2006 nodes.

PDDL problem description loaded: 
	Domain: VIRTUALHOME
	Problem: RELAX_WATCH_TV
	#Actions: 2006
	#Fluents: 1804
Landmarks found: 4
Starting search with IW (time budget is 60 secs)...
rel_plan size: 17
#RP_fluents 194
Caption
{#goals, #UNnachieved,  #Achieved} -> IW(max_w)

{4/4/0}:IW(1) -> [2][3][4]rel_plan size: 14
#RP_fluents 215
{4/3/1}:IW(1) -> [2][3][4]rel_plan size: 11
#RP_fluents 118
{4/2/2}:IW(1) -> [2][3][4][5][6]rel_plan size: 6
#RP_fluents 99
{4/1/3}:IW(1) -> [2][3][4][5][6][7]rel_plan size: 0
#RP_fluents 0Plan found with cost: 17
Total time: 9.89268
Nodes generated during search: 20647
Nodes expanded during search: 11034
IW search completed


In [16]:
steps = []
print('Goals: ', goal)
f = open('./example_out')
for line in f:
    steps.append(line)

steps

Goals:  (and (inside clothespants_387 clothespile_413) (inside plate_195 kitchencabinet_235) (grabbed character_1 towel_68) (grabbed character_2 cereal_335))


['(WALK BATHROOM_11 CHARACTER_1)\n',
 '(WALK TOWEL_68 CHARACTER_1)\n',
 '(GRAB TOWEL_68 CHARACTER_1)\n',
 '(WALK KITCHEN_207 CHARACTER_2)\n',
 '(WALK CEREAL_335 CHARACTER_2)\n',
 '(GRAB CEREAL_335 CHARACTER_2)\n',
 '(WALK KITCHENCABINET_235 CHARACTER_2)\n',
 '(OPEN KITCHENCABINET_235 CHARACTER_2)\n',
 '(WALK PLATE_195 CHARACTER_2)\n',
 '(GRAB PLATE_195 CHARACTER_2)\n',
 '(PUTIN PLATE_195 KITCHENCABINET_235 CHARACTER_2)\n',
 '(WALK LIVINGROOM_336 CHARACTER_1)\n',
 '(WALK CLOTHESPANTS_387 CHARACTER_1)\n',
 '(GRAB CLOTHESPANTS_387 CHARACTER_1)\n',
 '(WALK CLOTHESPILE_413 CHARACTER_1)\n',
 '(OPEN CLOTHESPILE_413 CHARACTER_1)\n',
 '(PUTIN CLOTHESPANTS_387 CLOTHESPILE_413 CHARACTER_1)\n']

In [17]:
count = 0
method = {}
for key in tasks.keys():


    content = ""
    outputfile = "./sample_env/out_problems/file_0.pddl"
    goal = key
    count+=1
    print(count, key)
    flag = False

    f = open('/content/PDDLMultiAgent/sample_env/out_problems/file_0.pddl', 'r')
    for line in f:
      #print(line)
      content += line if not flag else ""
      if line.startswith('(:goal'):
        flag = True 
      if flag and line.startswith(')'):
        flag = False
        content += goal + '\n'
        content += ')\n'
        
    wf = open(outputfile, 'w')
    wf.write(content)
    wf.close()
    domain_name='./domain.pddl'
    problem_name='./sample_env/out_problems/file_0.pddl'
    output_dir = "./example_out"

    ! ./LAPKT-public/planners/siw_plus-then-bfs_f-ffparser/siw-then-bfsf --domain $domain_name --problem $problem_name --output $output_dir



    steps = []
    f = open('./example_out')
    for line in f:
      steps.append(line)

    method[key] = steps


    



1 (and  (ontop hanger_393 bathroomcabinet_49) (inside clothespile_398 bookshelf_371) (grabbed character_1 cutleryfork_280) (grabbed character_2 hanger_121))


task contains conditional effects. turning off state domination.

 --- OK.
 Match tree built with 2006 nodes.

PDDL problem description loaded: 
	Domain: VIRTUALHOME
	Problem: RELAX_WATCH_TV
	#Actions: 2006
	#Fluents: 1804
Landmarks found: 4
Starting search with IW (time budget is 60 secs)...
rel_plan size: 19
#RP_fluents 242
Caption
{#goals, #UNnachieved,  #Achieved} -> IW(max_w)

{4/4/0}:IW(1) -> [2][3]rel_plan size: 17
#RP_fluents 227
{4/3/1}:IW(1) -> [2][3][4]rel_plan size: 11
#RP_fluents 122
{4/2/2}:IW(1) -> [2][3][4][5][6][7]rel_plan size: 5
#RP_fluents 100
{4/1/3}:IW(1) -> [2][3][4][5][6]rel_plan size: 0
#RP_fluents 0Plan found with cost: 16
Total time: 12.4486
Nodes generated during search: 23506
Nodes expanded during search: 13223
IW search completed
2 (and  (ontop hanger_115 plate_279) (inside hanger_393 clothespile_398

In [18]:
method

{'(and  (ontop bananas_317 bathroomcabinet_49) (inside clothespants_133 clothespile_395) (grabbed character_1 cutleryfork_280) (grabbed character_2 clothespile_395))': ['(WALK CUTLERYFORK_280 CHARACTER_1)\n',
  '(GRAB CUTLERYFORK_280 CHARACTER_1)\n',
  '(WALK BATHROOM_11 CHARACTER_2)\n',
  '(WALK LIVINGROOM_336 CHARACTER_2)\n',
  '(WALK CLOTHESPILE_395 CHARACTER_2)\n',
  '(GRAB CLOTHESPILE_395 CHARACTER_2)\n',
  '(OPEN CLOTHESPILE_395 CHARACTER_2)\n',
  '(WALK BANANAS_317 CHARACTER_1)\n',
  '(GRAB BANANAS_317 CHARACTER_1)\n',
  '(WALK BATHROOM_11 CHARACTER_1)\n',
  '(WALK BATHROOMCABINET_49 CHARACTER_1)\n',
  '(PUT BANANAS_317 BATHROOMCABINET_49 CHARACTER_1)\n',
  '(WALK BEDROOM_74 CHARACTER_2)\n',
  '(WALK CLOSET_129 CHARACTER_2)\n',
  '(OPEN CLOSET_129 CHARACTER_2)\n',
  '(WALK CLOTHESPANTS_133 CHARACTER_2)\n',
  '(GRAB CLOTHESPANTS_133 CHARACTER_2)\n',
  '(PUTIN CLOTHESPANTS_133 CLOTHESPILE_395 CHARACTER_2)\n'],
 '(and  (ontop bananas_317 bathroomcabinet_49) (inside mug_196 clothesp

In [19]:
with open("method.json", "w") as outfile:
    json.dump(method, outfile)

## NOT NEEDED

In [200]:
domain_name='./domain.pddl'
problem_name='./sample_env/out_problems/file_0.pddl'
output_dir = "./example_out"

In [197]:
! ./LAPKT-public/planners/siw_plus-then-bfs_f-ffparser/siw-then-bfsf --domain $domain_name --problem $problem_name --output $output_dir



task contains conditional effects. turning off state domination.

 --- OK.
 Match tree built with 2006 nodes.

PDDL problem description loaded: 
	Domain: VIRTUALHOME
	Problem: RELAX_WATCH_TV
	#Actions: 2006
	#Fluents: 1802
Landmarks found: 4
Starting search with IW (time budget is 60 secs)...
rel_plan size: 16
#RP_fluents 193
Caption
{#goals, #UNnachieved,  #Achieved} -> IW(max_w)

{4/4/0}:IW(1) -> [2][3][4]rel_plan size: 15
#RP_fluents 220
{4/3/1}:IW(1) -> [2][3][4]rel_plan size: 11
#RP_fluents 126
{4/2/2}:IW(1) -> [2][3][4][5][6]rel_plan size: 7
#RP_fluents 105
{4/1/3}:IW(1) -> [2][3][4][5][6][7][8]rel_plan size: 0
#RP_fluents 0Plan found with cost: 18
Total time: 18.3431
Nodes generated during search: 30186
Nodes expanded during search: 17581
IW search completed


In [198]:
f = open('./example_out')
for line in f:
    print(line)

(WALK KITCHEN_207 CHARACTER_2)

(WALK MUG_196 CHARACTER_2)

(GRAB MUG_196 CHARACTER_2)

(WALK LIVINGROOM_336 CHARACTER_1)

(WALK CLOTHESPILE_413 CHARACTER_1)

(GRAB CLOTHESPILE_413 CHARACTER_1)

(OPEN CLOTHESPILE_413 CHARACTER_1)

(WALK KITCHEN_207 CHARACTER_1)

(WALK CEREAL_335 CHARACTER_1)

(GRAB CEREAL_335 CHARACTER_1)

(PUTIN CEREAL_335 CLOTHESPILE_413 CHARACTER_1)

(WALK BATHROOM_11 CHARACTER_2)

(WALK TOWEL_68 CHARACTER_2)

(GRAB TOWEL_68 CHARACTER_2)

(WALK LIVINGROOM_336 CHARACTER_2)

(WALK CLOTHESPILE_402 CHARACTER_2)

(OPEN CLOTHESPILE_402 CHARACTER_2)

(PUTIN TOWEL_68 CLOTHESPILE_402 CHARACTER_2)



## Parse Generated Data

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
gdrive_root = "/content/drive"
data_dir = os.path.join(gdrive_root, 'MyDrive' ,'Dataset', 'lm_vh')
train_dir = os.path.join(data_dir, 'gather_data_actiongraph_train.json')
test_dir = os.path.join(data_dir, 'gather_data_actiongraph_test.json')
assert os.path.exists(data_dir), 'Check the path to your root directory'
assert os.path.exists(train_dir), 'Check the path to your root directory'
assert os.path.exists(test_dir), 'Check the path to your root directory'

In [ ]:
!pip install ijson

In [ ]:
import json
import ijson

#For training data
res = []
with open(train_dir, "r") as f:
    for record in ijson.items(f, "train_data.item"):
        res += [{'env': record['env_id'], 'goal': record['goal'], 'script':record['valid_action_with_walk']}]
        print(record['name'])

parsed_train_dir = os.path.join(data_dir, 'parsed_train.json')
with open(parsed_train_dir, 'w') as fp:
    json.dump(res, fp)

In [ ]:
#for test data
res = []
with open(test_dir, "r") as f:
    for record in ijson.items(f, "test_data.item"):
        res += [{'env': record['env_id'], 'goal': record['goal'], 'script':record['valid_action_with_walk']}]
        print(record['name'])
        
parsed_test_dir = os.path.join(data_dir, 'parsed_test.json')
with open(parsed_test_dir, 'w') as fp:
    json.dump(res, fp)